# 数据集准备
## HR→LR 合成退化流水线
（高斯/运动模糊 → 下采样 ×2/×3/×4 → 噪声/JPEG → 可选划痕叠加）


In [ ]:
# ============== DIV2K 数据集下载（Linux 云服务器使用） ==============
# 在 Linux 服务器上运行此单元格下载 DIV2K HR 数据集
# 下载完成后会自动解压并整理目录结构

import os
import subprocess
import zipfile
from pathlib import Path

# 配置路径（根据实际情况修改）
DATA_ROOT = Path("../data")  # 相对于 notebook 的数据目录

# 创建目录
DATA_ROOT.mkdir(parents=True, exist_ok=True)
os.chdir(DATA_ROOT)

# DIV2K 下载链接
URLS = {
    "train_HR": "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip",
    "valid_HR": "http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip",
}

def download_and_extract(name, url):
    zip_file = f"DIV2K_{name}.zip"
    extract_dir = f"DIV2K_{name}"
    target_dir = "train_HR" if "train" in name else "val_HR"
    
    # 检查是否已存在
    if Path(target_dir).exists() and len(list(Path(target_dir).glob("*.png"))) > 50:
        print(f"[SKIP] {target_dir} already exists with images")
        return
    
    # 下载
    if not Path(zip_file).exists():
        print(f"[DL] Downloading {name}...")
        # 使用 wget（Linux 常用）或 curl
        result = subprocess.run(
            ["wget", "-c", "-O", zip_file, url],
            capture_output=False
        )
        if result.returncode != 0:
            # 尝试 curl
            subprocess.run(["curl", "-L", "-C", "-", "-o", zip_file, url])
    else:
        print(f"[OK] {zip_file} already downloaded")
    
    # 解压
    if not Path(extract_dir).exists():
        print(f"[ZIP] Extracting {zip_file}...")
        with zipfile.ZipFile(zip_file, 'r') as zf:
            zf.extractall(".")
    
    # 重命名到目标目录
    if Path(extract_dir).exists() and not Path(target_dir).exists():
        Path(extract_dir).rename(target_dir)
        print(f"[OK] Moved to {target_dir}")
    
    print(f"[DONE] {target_dir}: {len(list(Path(target_dir).glob('*.png')))} images")

# 执行下载
for name, url in URLS.items():
    download_and_extract(name, url)

# 回到原目录
os.chdir("..")
print("\n[COMPLETE] Dataset ready. Run next cells to generate LR images.")

In [ ]:
# 读取 data 中 HR 并做快速检查（在配置单元前执行）
import os
from pathlib import Path
from PIL import Image

# 自动检测路径（兼容 Windows 和 Linux）
NOTEBOOK_DIR = Path(os.getcwd())
if (NOTEBOOK_DIR / "data").exists():
    ROOT = NOTEBOOK_DIR / "data"
elif (NOTEBOOK_DIR.parent / "data").exists():
    ROOT = NOTEBOOK_DIR.parent / "data"
else:
    # 手动指定（根据实际情况修改）
    ROOT = Path("../data")

ROOT = str(ROOT.resolve())
EXTS = ('.png', '.jpg', '.jpeg', '.bmp')

print(f"Data root: {ROOT}")

for sp in ['train', 'val']:
    hr_dir = os.path.join(ROOT, f'{sp}_HR')
    exists = os.path.isdir(hr_dir)
    files = [f for f in os.listdir(hr_dir) if f.lower().endswith(EXTS)] if exists else []
    print(f'{sp} HR: {hr_dir} | exists: {exists} | count: {len(files)}')

In [ ]:
import os, io, math, random, hashlib
from typing import Tuple, List, Optional
import numpy as np
from PIL import Image, ImageFilter
from pathlib import Path

# ============== 配置（自动检测路径，兼容 Windows/Linux） ==============
# 优先使用上一个单元格检测到的 ROOT，否则自动检测
if 'ROOT' in dir():
    DATA_ROOT = ROOT
else:
    NOTEBOOK_DIR = Path(os.getcwd())
    if (NOTEBOOK_DIR / "data").exists():
        DATA_ROOT = str((NOTEBOOK_DIR / "data").resolve())
    elif (NOTEBOOK_DIR.parent / "data").exists():
        DATA_ROOT = str((NOTEBOOK_DIR.parent / "data").resolve())
    else:
        DATA_ROOT = "../data"

SPLITS    = ['train', 'val']
SCALES    = [4]             # 可设为 [2,3,4]；当前默认只做 x4
SEED      = 42
OVERLAY_DIR = os.path.join(DATA_ROOT, 'overlays')

# 参数区间（可调）
BLUR_GAUSS_SIGMA = (0.0, 1.6)
MOTION_KSIZE     = (3, 11)
MOTION_ANGLE     = (0, 180)
GAUSS_NOISE_STD  = (0.0, 6.0)
SP_NOISE_PROB    = (0.0, 0.01)
JPEG_QUALITY     = (60, 95)
OVERLAY_PROB     = 0.25
OVERLAY_ALPHA    = (0.1, 0.5)
OVERLAY_SCALE    = (0.5, 1.5)

print('DATA_ROOT =', DATA_ROOT)

def list_images(folder, exts=('.png','.jpg','.jpeg','.bmp')):
    if not os.path.isdir(folder):
        return []
    names = [f for f in os.listdir(folder) if f.lower().endswith(exts)]
    names.sort()
    return names

def imread_rgb(path: str) -> Image.Image:
    img = Image.open(path)
    if img.mode != 'RGB':
        img = img.convert('RGB')
    return img

def per_image_seed(global_seed: int, split: str, scale: int, stem: str) -> int:
    s = f'{global_seed}|{split}|x{scale}|{stem}'
    h = hashlib.sha256(s.encode('utf-8')).hexdigest()
    return int(h[:8], 16)

def _shift_array_edge(arr: np.ndarray, dx: int, dy: int) -> np.ndarray:
    h, w = arr.shape[:2]
    pad_l = max(0, dx); pad_r = max(0, -dx)
    pad_t = max(0, dy); pad_b = max(0, -dy)
    arr_pad = np.pad(arr, ((pad_t, pad_b), (pad_l, pad_r), (0,0)), mode='edge')
    y0 = pad_t - dy; x0 = pad_l - dx
    return arr_pad[y0:y0+h, x0:x0+w, :]

def apply_motion_blur(img: Image.Image, ksize: int, angle_deg: float) -> Image.Image:
    ksize = max(3, int(ksize) | 1)
    angle = math.radians(angle_deg)
    dx = math.cos(angle); dy = math.sin(angle)
    arr = np.asarray(img).astype(np.float32)
    acc = np.zeros_like(arr, dtype=np.float32)
    n = ksize; center = (n - 1) / 2.0
    for i in range(n):
        offset = i - center
        sx = int(round(offset * dx)); sy = int(round(offset * dy))
        shifted = _shift_array_edge(arr, sx, sy)
        acc += shifted
    acc /= n
    acc = np.clip(acc, 0, 255).astype(np.uint8)
    return Image.fromarray(acc)

def apply_gaussian_blur(img: Image.Image, sigma: float) -> Image.Image:
    if sigma <= 0.01:
        return img
    return img.filter(ImageFilter.GaussianBlur(radius=sigma))

def downsample(img: Image.Image, scale: int) -> Image.Image:
    w, h = img.size
    return img.resize((max(1, w // scale), max(1, h // scale)), Image.Resampling.BICUBIC)

def add_gaussian_noise(img: Image.Image, std: float) -> Image.Image:
    if std <= 0.01:
        return img
    arr = np.asarray(img).astype(np.float32)
    noise = np.random.normal(0, std, arr.shape)
    arr = np.clip(arr + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(arr)

def add_salt_pepper_noise(img: Image.Image, prob: float) -> Image.Image:
    if prob <= 1e-5:
        return img
    arr = np.asarray(img).astype(np.uint8)
    mask = np.random.rand(*arr.shape[:2])
    salt = mask < (prob / 2)
    pepper = (mask >= (prob / 2)) & (mask < prob)
    arr[salt] = 255
    arr[pepper] = 0
    return Image.fromarray(arr)

def add_jpeg_artifact(img: Image.Image, quality: int) -> Image.Image:
    quality = int(max(1, min(quality, 100)))
    with io.BytesIO() as buf:
        img.save(buf, format='JPEG', quality=quality)
        buf.seek(0)
        return Image.open(buf).convert('RGB')

def try_overlay(img: Image.Image, rng: random.Random):
    if not os.path.isdir(OVERLAY_DIR) or rng.random() > OVERLAY_PROB:
        return img, None, None
    cand = [f for f in os.listdir(OVERLAY_DIR) if f.lower().endswith(('.png', '.webp'))]
    if not cand:
        return img, None, None
    ov_name = rng.choice(cand)
    ov = Image.open(os.path.join(OVERLAY_DIR, ov_name)).convert('RGBA')
    s = rng.uniform(*OVERLAY_SCALE)
    new_w = max(1, int(img.width * s))
    new_h = max(1, int(ov.height * (new_w / max(1, ov.width))))
    ov = ov.resize((new_w, new_h), Image.Resampling.BICUBIC)
    x = rng.randint(-new_w // 2, img.width - 1)
    y = rng.randint(-new_h // 2, img.height - 1)
    alpha = rng.uniform(*OVERLAY_ALPHA)
    r, g, b, a = ov.split()
    a = a.point(lambda v: int(v * alpha))
    ov = Image.merge('RGBA', (r, g, b, a))
    base = img.convert('RGBA')
    base.alpha_composite(ov, dest=(x, y))
    return base.convert('RGB'), ov_name, alpha

In [ ]:
def degrade_one(hr_img: Image.Image, scale: int, rng: random.Random):
    log = {}

    # 1) 模糊：高斯/运动 二选一
    if rng.random() < 0.5:
        sigma = rng.uniform(*BLUR_GAUSS_SIGMA)
        lr = apply_gaussian_blur(hr_img, sigma)
        log['blur_type'] = 'gaussian'
        log['blur_sigma'] = round(sigma, 4)
        log['motion_ksize'] = ''
        log['motion_angle'] = ''
    else:
        ksz = rng.randrange(MOTION_KSIZE[0] | 1, (MOTION_KSIZE[1] | 1) + 1, 2)
        ang = rng.uniform(*MOTION_ANGLE)
        lr = apply_motion_blur(hr_img, ksz, ang)
        log['blur_type'] = 'motion'
        log['blur_sigma'] = ''
        log['motion_ksize'] = int(ksz)
        log['motion_angle'] = round(ang, 2)

    # 2) 下采样
    lr = downsample(lr, scale)
    log['down_scale'] = int(scale)

    # 3) 噪声：无 / 高斯 / 椒盐（按 1/3 概率）
    r = rng.random()
    if r < 1/3:
        log['noise_type'] = 'none'
        log['noise_param'] = ''
    elif r < 2/3:
        std = rng.uniform(*GAUSS_NOISE_STD)
        lr = add_gaussian_noise(lr, std)
        log['noise_type'] = 'gaussian'
        log['noise_param'] = round(std, 4)
    else:
        prob = rng.uniform(*SP_NOISE_PROB)
        lr = add_salt_pepper_noise(lr, prob)
        log['noise_type'] = 'salt_pepper'
        log['noise_param'] = round(prob, 6)

    # 4) JPEG 伪影
    q = rng.randint(*JPEG_QUALITY)
    lr = add_jpeg_artifact(lr, q)
    log['jpeg_quality'] = int(q)

    # 5) 可选划痕/污渍叠加
    lr, ov_name, ov_alpha = try_overlay(lr, rng)
    log['overlay'] = ov_name or ''
    log['overlay_alpha'] = round(ov_alpha, 3) if ov_alpha is not None else ''

    return lr, log


def make_lr_for_split(split: str, scale_list: List[int]):
    hr_dir = os.path.join(DATA_ROOT, f'{split}_HR')
    files = list_images(hr_dir)
    if not files:
        print(f'⚠️ 未发现 {hr_dir} 图像，跳过 {split}')
        return {s:0 for s in scale_list}

    lr_dir = os.path.join(DATA_ROOT, f'{split}_LR')
    os.makedirs(lr_dir, exist_ok=True)

    results = {}
    for scale in scale_list:
        print(f'开始处理 {split} x{scale}: 共 {len(files)} 张')
        cnt = 0
        for i, name in enumerate(files, 1):
            stem, ext = os.path.splitext(name)
            hr = imread_rgb(os.path.join(hr_dir, name))

            s = per_image_seed(SEED, split, scale, stem)
            rng = random.Random(s)
            np.random.seed(s)

            lr, log = degrade_one(hr, scale, rng)
            out_name = f'{stem}x{scale}.png'
            lr.save(os.path.join(lr_dir, out_name))

            if i % 50 == 0 or i == len(files):
                print(f'  已完成 {i}/{len(files)}')
            cnt += 1

        print(f'✅ {split} x{scale} 完成：生成 {cnt} 张')
        results[scale] = cnt
    return results


total = 0
for sp in SPLITS:
    res = make_lr_for_split(sp, SCALES)
    total += sum(res.values())
print(f'🎉 全部完成，总计生成 LR：{total} 张')

# 可选：快速检查 {split}_LR 数量与命名
check_scale = SCALES[0] if len(SCALES) == 1 else None
if check_scale is not None:
    for sp in SPLITS:
        hr_dir = os.path.join(DATA_ROOT, f'{sp}_HR')
        lr_dir = os.path.join(DATA_ROOT, f'{sp}_LR')
        hr_files = list_images(hr_dir)
        lr_files = list_images(lr_dir)
        print(sp, 'HR:', len(hr_files), 'LR:', len(lr_files))
        for n in hr_files[:5]:
            stem, _ = os.path.splitext(n)
            expect = f'{stem}x{check_scale}.png'
            print('  ', n, '→', expect, '存在' if expect in lr_files else '缺失')